# Bibtex to DF and tidy

**One off code for data rationalization and normalization.**

* df
* author list
* publisher list
* journal list
* files
* normalization

item 241 has weird annot


## Provenance
* Created new 2025-05-16 at Bignor!

In [8]:
from pathlib import Path 
import pandas as pd
import re
from functools import partial
import pprint as pp

import Levenshtein

import scratch as sc
import great2_bibtex as scbt

from greater_tables import GT
def safeyear(s):
    try:
        return f'{int(s)}'
    except:
        return s
fGT = partial(GT, large_ok=True, formatters={'index': str, 'year': safeyear})
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 15 ms (started: 2025-05-21 09:46:12 +01:00)


## Manual read and manual inspection

In [38]:
p = Path('books.bib')
p = Path('library.bib')
p.exists()

True

time: 16 ms (started: 2025-05-21 10:08:58 +01:00)


In [39]:
b = sc.Bib2df(p, False)
len(b.df)

uber regex sub found n = 517 replacements


6118

time: 1.69 s (started: 2025-05-21 10:08:58 +01:00)


In [40]:
b.df.head(2).T

,1,2
type,article,book
tag,Schmidt2003,Bahnemann2015
abstract,It is shown that the arbitrage free portfolio ...,NaN
author,"Schmidt, Anatoly B.","Bahnemann, David"
doi,10.1016/S0378-4371(02)01551-0,NaN
file,:C\:/S/Library/Schmidt/2003_The true invariant...,:C\:/S/Library/Bahnemann/2015_Distributions fo...
issn,03784371,NaN
journal,Physica A: Statistical Mechanics and its Appli...,NaN
keywords,"Arbitrage free portfolio,Option pricing",NaN
pages,535--538,NaN


time: 15 ms (started: 2025-05-21 10:09:02 +01:00)


In [41]:
# filled values for each column  
b.contents()

,column,nonna,distinct
0,type,6118,10
1,tag,6118,6076
2,abstract,2518,2375
3,author,6113,4856
4,doi,2687,2538
5,file,5951,5917
6,issn,2425,826
7,journal,3475,1224
8,keywords,2999,2052
9,pages,3885,2748


time: 250 ms (started: 2025-05-21 10:09:06 +01:00)


In [ ]:
for f in b.contents().column: 
    display(fGT(b.df[f].value_counts().sort_values()[::-1].head(30), caption=f))
    print()

# Porting Menedeley

In [ ]:
b = sc.Bib2df(p, False)

In [ ]:
b.port_mendeley_file()

In [ ]:
# run fillna = False
df = b.contents(True)

In [6]:
base_cols = ['tag', 'type', 'author', 'title', 'year', 'journal']

time: 0 ns (started: 2025-05-21 09:43:46 +01:00)


In [377]:
b.df.loc[b.df.file.isna(), base_cols]

,tag,type,author,title,year
38,keynesgeneral,book,"Keynes, John Maynard","{General theory of employment, interest and mo...",1936
53,Mildenhall2018c,article,"Mildenhall, Stephen J",{Every law invariant coherent risk measure has...,2018
75,Eling2017,article,"Eling, Martin and Jia, Ruo",{Recent Research Developments Affecting Nonlif...,2017
114,daykin94,book,"Daykin, C. D. and Pentikainen, T. and Pesonen, M.",{Practical Risk Theory for Actuaries},1994
133,Crouhy2010,article,"Crouhy, Michel and Galai, Dan and Mark, Robert",{Risk Exposures},2010
...,...,...,...,...,...
5865,hull01,book,"Hiriart-Urruty, Jean-Baptiste and Lemar{\'{e}}...",{Fundamentals of Convex Analysis},2001
5931,eeckhoudtgs04,book,"Eeckhoudt, Louis and Gollier, Christian and Sc...",{Economic and Financial Decisions under Risk},2005
6006,Strain1997,book,"Strain, Robert W.",{Reinsurance},1997
6039,jacob02,book,"Jacob, Niels",{Pseduo Differential Operators & Markov Proces...,2002


time: 16 ms (started: 2025-05-18 22:28:54 +01:00)


In [381]:
b = sc.Bib2df(p, True)

time: 312 ms (started: 2025-05-18 22:31:10 +01:00)


In [ ]:
fGT(b.df.loc[b.df['mendeley-tags'].str.contains('Book', case=False), base_cols + ['mendeley-tags', 'file']])

6118

time: 15 ms (started: 2025-05-18 21:18:34 +01:00)


## File demographics

### Searching

In [ ]:
b.df.loc[b.df.author.str.contains('Bloch|Deligne', regex=True, case=False).fillna(False)]

### Duplicated tag

In [10]:
base_cols = ['tag', 'type', 'author', 'title', 'year', 'journal', 'file', 'publisher']
len(base_cols)

8

time: 16 ms (started: 2025-05-21 09:47:17 +01:00)


In [37]:
for t in sorted(b.df.loc[b.df.tag.duplicated(keep=False), base_cols].tag.value_counts().keys()):
    bit = b.df.drop(columns=['abstract', 'annote']).query('tag == @t').T.dropna(how='all')
    bit_eq = (bit.eq(bit.iloc[:, 0], axis=0).all(axis=1)).astype(int)
    bit['union'] = bit.apply(lambda x: x.iloc[0] if pd.isna(x.iloc[1]) else x.iloc[1], axis=1)
    bit['eq'] = bit_eq
    display(fGT(bit.loc[~bit.union.isna()].fillna(''), caption=t))

index,742,2288,union,eq
type,book,book,book,1
tag,Aliprantis2006,Aliprantis2006,Aliprantis2006,1
author,"Aliprantis, Charalambos D and Burkinshaw, Owen","Aliprantis, Charalambos D and Border, Kim","Aliprantis, Charalambos D and Border, Kim",0
doi,10.1007/978-94-015-9349-6_3,10.1007/3-540-29587-9,10.1007/3-540-29587-9,0
file,":C\:/S/Library/Aliprantis, Burkinshaw/2006_Positive Operators.pdf:pdf",":C\:/S/Library/Aliprantis, Border/2006_Infinite Dimensional Analysis A Hitchhiker's Guide.pdf:pdf",":C\:/S/Library/Aliprantis, Border/2006_Infinite Dimensional Analysis A Hitchhiker's Guide.pdf:pdf",0
keywords,,Book,Book,0
pages,,704,704,0
title,{Positive Operators},{Infinite Dimensional Analysis: A Hitchhiker's Guide},{Infinite Dimensional Analysis: A Hitchhiker's Guide},0
year,"2,006","2,006","2,006",1
isbn,"9,781,402,050,077","3,540,326,960","3,540,326,960",0


index,418,5541,union,eq
type,article,techreport,techreport,0
tag,AonBenfield2016,AonBenfield2016,AonBenfield2016,1
author,"Mildenhall, Stephen J",{Aon Benfield},{Aon Benfield},0
file,:C\:/S/Library/Mildenhall/2016_Insurance Risk Study The First Ten Years.pdf:pdf,:C\:/S/Library/Aon Benfield/2016_Cyber Opportunity and Future Growth.pdf:pdf,:C\:/S/Library/Aon Benfield/2016_Cyber Opportunity and Future Growth.pdf:pdf,0
keywords,"Aon,Mildenhall",Aon,Aon,0
title,{Insurance Risk Study: The First Ten Years},{Cyber Opportunity and Future Growth},{Cyber Opportunity and Future Growth},0
year,"2,016","2,016","2,016",1
mendeley-tags,"Aon,Mildenhall",Aon,Aon,0


index,1793,3225,union,eq
type,article,unpublished,unpublished,0
tag,Avanzi2011,Avanzi2011,Avanzi2011,1
author,"Avanzi, Benjamin","Avanzi, Benjamin and Cassar, Luke C and Wong, Bernard","Avanzi, Benjamin and Cassar, Luke C and Wong, Bernard",0
doi,10.2143/AST.41.2.2136990,,10.2143/AST.41.2.2136990,0
file,:C\:/S/Library/Avanzi/2011_Optimal dividends and capital injections in the dual model with diffusions.pdf:pdf,":C\:/S/Library/Avanzi, Cassar, Wong/2011_Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy C.pdf:pdf",":C\:/S/Library/Avanzi, Cassar, Wong/2011_Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy C.pdf:pdf",0
issn,05150361 17831350,,05150361 17831350,0
journal,ASTIN Bulletin,,ASTIN Bulletin,0
keywords,,"c46,c51,c52,compound poisson process,dependence,g22,insurance,jel codes,l{\'{e}}vy copula,real data","c46,c51,c52,compound poisson process,dependence,g22,insurance,jel codes,l{\'{e}}vy copula,real data",0
pages,611--644,,611--644,0
title,{Optimal dividends and capital injections in the dual model with diffusions},{Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas},{Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas},0


index,2023,2938,union,eq
type,article,article,article,1
tag,Babbel1997,Babbel1997,Babbel1997,1
author,"Babbel, David F.","Babbel, David F","Babbel, David F",0
journal,North American Actuarial Journal,North American Actuarial Journal,North American Actuarial Journal,1
pages,122--125,122--123,122--123,0
title,"{Discussion of ``Two Paradigms for The Market Value of Liabilities, Robert R. Reitano''}",{Review of Two Paradigms for the Market Value of Liabilities by Reitano},{Review of Two Paradigms for the Market Value of Liabilities by Reitano},0
volume,1,1(4),1(4),0
year,"1,997","1,997","1,997",1
number,4,,4,0


index,3857,4082,union,eq
type,article,article,article,1
tag,Bailey1967,Bailey1967,Bailey1967,1
author,"Bailey, Robert A","Bailey, Robert A","Bailey, Robert A",1
file,:C\:/S/Library/Bailey/1967_Underwriting Profit From Investments.pdf:pdf,,:C\:/S/Library/Bailey/1967_Underwriting Profit From Investments.pdf:pdf,0
journal,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,1
keywords,UNDERWRITING PROFIT FROM INVESTMENTS,,UNDERWRITING PROFIT FROM INVESTMENTS,0
pages,1--8,1--8,1--8,1
title,{Underwriting Profit From Investments},{Underwriting Profits From Investments},{Underwriting Profits From Investments},0
volume,LIV,LIV,LIV,1
year,"1,967","1,967","1,967",1


...

index,1732,2699,union,eq
type,article,article,article,1
tag,Vinogradov2004,Vinogradov2004,Vinogradov2004,1
author,"Vinogradov, Vladimir","Vinogradov, Vladimir","Vinogradov, Vladimir",1
doi,10.31390/cosa.2.2.02,10.1081/STA-120029821,10.1081/STA-120029821,0
file,:C\:/S/Library/Vinogradov/2004_Properties of certain L{\'{e}}vy and geometric L{\'{e}}vy processes.pdf:pdf;:C\:/S/Library/Vinogradov/2004_Properties of certain L{\'{e}}vy and geometric L{\'{e}}vy processes.pdf:pdf,:C\:/S/Library/Vinogradov/2004_On the power-variance family of probability distributions.pdf:pdf,:C\:/S/Library/Vinogradov/2004_On the power-variance family of probability distributions.pdf:pdf,0
issn,0973-9599,"3,610,926","3,610,926",0
journal,Communications on Stochastic Analysis,Communications in Statistics - Theory and Methods,Communications in Statistics - Theory and Methods,0
keywords,,"Cumulant,Decomposition criterion,Indecomposable component,Power-variance family,Tweedie law,Weak convergence","Cumulant,Decomposition criterion,Indecomposable component,Power-variance family,Tweedie law,Weak convergence",0
pages,,1007--1029,1007--1029,0
title,{Properties of certain L{\'{e}}vy and geometric L{\'{e}}vy processes},{On the power-variance family of probability distributions},{On the power-variance family of probability distributions},0


index,1185,1961,union,eq
type,article,article,article,1
tag,Wang2000,Wang2000,Wang2000,1
author,"Wang, Shaun S.","Wang, Shaun S.","Wang, Shaun S.",1
doi,,10.2307/253675,10.2307/253675,0
file,:C\:/S/Library/Wang/2000_A class of distorion operators for pricing financial and insurance risks.pdf:pdf,:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks.pdf:pdf;:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks(2).pdf:pdf,:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks.pdf:pdf;:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks(2).pdf:pdf,0
issn,,"224,367","224,367",0
journal,Journal of Risk and Insurance,The Journal of Risk and Insurance,The Journal of Risk and Insurance,0
keywords,WangSS,WangSS,WangSS,1
pages,15--36,15--36,15--36,1
title,{A class of distorion operators for pricing financial and insurance risks},{A Class of Distortion Operators for Pricing Financial and Insurance Risks},{A Class of Distortion Operators for Pricing Financial and Insurance Risks},0


index,1225,1356,union,eq
type,article,article,article,1
tag,Wang2018,Wang2018,Wang2018,1
author,"Wang, Ruodu and Wei, Yunran and Willmot, Gordon","Wang, Ruodu and Wei, Yunran and Willmot, Gordon","Wang, Ruodu and Wei, Yunran and Willmot, Gordon",1
doi,10.2139/ssrn.2956962,10.2139/ssrn.2956962,10.2139/ssrn.2956962,1
file,":C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals(2).pdf:pdf;:C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals.pdf:pdf",":C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals(2).pdf:pdf",":C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals(2).pdf:pdf",0
journal,SSRN Electronic Journal,SSRN Electronic Journal,SSRN Electronic Journal,1
keywords,"PMIR,WangR,choquet integrals,comonotonicity,risk aggregation,risk functionals,robustness","WangR,choquet integrals,comonotonicity,risk aggregation,risk functionals,robustness","WangR,choquet integrals,comonotonicity,risk aggregation,risk functionals,robustness",0
title,"{Characterization, Robustness and Aggregation of Signed Choquet Integrals}","{Characterization, Robustness and Aggregation of Signed Choquet Integrals}","{Characterization, Robustness and Aggregation of Signed Choquet Integrals}",1
year,"2,018","2,018","2,018",1
mendeley-tags,"PMIR,WangR",WangR,WangR,0


index,2488,5380,union,eq
type,article,article,article,1
tag,Yin2018,Yin2018,Yin2018,1
author,"Yin, Chuancun and Zhu, Dan","Yin, Chuancun and Zhu, Dan","Yin, Chuancun and Zhu, Dan",1
doi,,10.4236/jfrm.2018.71002,10.4236/jfrm.2018.71002,0
file,,":C\:/S/Library/Yin, Zhu/2018_New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR.pdf:pdf",":C\:/S/Library/Yin, Zhu/2018_New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR.pdf:pdf",0
issn,,2167-9533,2167-9533,0
journal,Journal of Financial Risk Management,Journal of Financial Risk Management,Journal of Financial Risk Management,1
keywords,,"additivity,coherent risk measure,comonotonicity,copula,distortion functions,distortion risk measure,extreme value theory,gluevar,maximum domain of attraction,spectral risk measure,super,tail distortion risk measure,tail sub,tvar","additivity,coherent risk measure,comonotonicity,copula,distortion functions,distortion risk measure,extreme value theory,gluevar,maximum domain of attraction,spectral risk measure,super,tail distortion risk measure,tail sub,tvar",0
pages,12--38,12--38,12--38,1
title,{No Title},{New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR},{New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR},0


time: 4.44 s (started: 2025-05-21 10:05:00 +01:00)


index,1316,5844,union,eq
type,article,article,article,1
tag,Lange1966,Lange1966,Lange1966,1
author,"Lange, Jeffrey T","Lange, Jeffrey T.","Lange, Jeffrey T.",0
file,:C\:/S/Library/Lange/1966_General Liability Insurance Ratemaking.pdf:pdf,:C\:/S/Library/Lange/1966_General Liability Insurance Ratemaking.pdf:pdf,:C\:/S/Library/Lange/1966_General Liability Insurance Ratemaking.pdf:pdf,1
journal,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,1
pages,26--53,26--53,26--53,1
title,{General liability insurance ratemaking},{General Liability Insurance Ratemaking},{General Liability Insurance Ratemaking},0
volume,LIII,nan,LIII,0
year,"1,966","1,966","1,966",1
url,nan,https://www.insureon.com/small-business-insurance/general-liability/cost,https://www.insureon.com/small-business-insurance/general-liability/cost,0


time: 125 ms (started: 2025-05-21 09:58:20 +01:00)


In [27]:
fGT(bit)

index,1316,5844,union,eq
type,article,article,article,1
tag,Lange1966,Lange1966,Lange1966,1
author,"Lange, Jeffrey T","Lange, Jeffrey T.","Lange, Jeffrey T.",0
file,:C\:/S/Library/Lange/1966_General Liability Insurance Ratemaking.pdf:pdf,:C\:/S/Library/Lange/1966_General Liability Insurance Ratemaking.pdf:pdf,:C\:/S/Library/Lange/1966_General Liability Insurance Ratemaking.pdf:pdf,1
journal,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,1
pages,26--53,26--53,26--53,1
title,{General liability insurance ratemaking},{General Liability Insurance Ratemaking},{General Liability Insurance Ratemaking},0
year,"1,966","1,966","1,966",1


time: 78 ms (started: 2025-05-21 09:57:41 +01:00)


In [12]:
fGT(b.df.loc[b.df.tag.duplicated(keep=False), base_cols].sort_values('tag') , tabs=[5, 10, 5, 15, 20, 3, 10, 20, 8])

index,tag,type,author,title,year,journal,file,publisher
2288,Aliprantis2006,book,"Aliprantis, Charalambos D and Border, Kim",{Infinite Dimensional Analysis: A Hitchhiker's Guide},2006,nan,":C\:/S/Library/Aliprantis, Border/2006_Infinite Dimensional Analysis A Hitchhiker's Guide.pdf:pdf",Springer Verlag
742,Aliprantis2006,book,"Aliprantis, Charalambos D and Burkinshaw, Owen",{Positive Operators},2006,nan,":C\:/S/Library/Aliprantis, Burkinshaw/2006_Positive Operators.pdf:pdf",Springer
418,AonBenfield2016,article,"Mildenhall, Stephen J",{Insurance Risk Study: The First Ten Years},2016,nan,:C\:/S/Library/Mildenhall/2016_Insurance Risk Study The First Ten Years.pdf:pdf,nan
5541,AonBenfield2016,techreport,{Aon Benfield},{Cyber Opportunity and Future Growth},2016,nan,:C\:/S/Library/Aon Benfield/2016_Cyber Opportunity and Future Growth.pdf:pdf,nan
1793,Avanzi2011,article,"Avanzi, Benjamin",{Optimal dividends and capital injections in the dual model with diffusions},2011,ASTIN Bulletin,:C\:/S/Library/Avanzi/2011_Optimal dividends and capital injections in the dual model with diffusions.pdf:pdf,nan
3225,Avanzi2011,unpublished,"Avanzi, Benjamin and Cassar, Luke C and Wong, Bernard",{Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas},2011,nan,":C\:/S/Library/Avanzi, Cassar, Wong/2011_Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy C.pdf:pdf",nan
2938,Babbel1997,article,"Babbel, David F",{Review of Two Paradigms for the Market Value of Liabilities by Reitano},1997,North American Actuarial Journal,nan,nan
2023,Babbel1997,article,"Babbel, David F.","{Discussion of ``Two Paradigms for The Market Value of Liabilities, Robert R. Reitano''}",1997,North American Actuarial Journal,nan,nan
3857,Bailey1967,article,"Bailey, Robert A",{Underwriting Profit From Investments},1967,Proceedings of the Casualty Actuarial Society,:C\:/S/Library/Bailey/1967_Underwriting Profit From Investments.pdf:pdf,nan
4082,Bailey1967,article,"Bailey, Robert A",{Underwriting Profits From Investments},1967,Proceedings of the Casualty Actuarial Society,nan,nan


time: 313 ms (started: 2025-05-21 09:47:59 +01:00)


### Distinct Journals

In [499]:
j = b.distinct('journal')
j[:10]

['116 Mich. L. Re.',
 '1Mathematical Intelligencer',
 '1Stochastic Processes and their Applications',
 '2003 IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2003. Proceedings.',
 '2012 International Conference on Systems and Informatics (ICSAI2012)',
 '2Variance',
 '30th Annual Symposium on Foundations of Computer Science',
 '31st International Congress of Actuaries',
 '34th International Conference on Machine Learning, ICML 2017',
 'A Modern Course on Statistical Distributions in Scientific Work']

time: 16 ms (started: 2025-05-19 20:33:54 +01:00)


In [506]:
jdf = b.journal_map_df()

Decode errors: len(self.last_decode) = 7
time: 906 ms (started: 2025-05-19 20:38:28 +01:00)


In [507]:
b.last_decode

['tex_to_unicode DECODE ERR | "Algorithms–ESA 2012      " | \'latin-1\' codec can\'t encode character \'\\u2013\' in position 10: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Journal of Economic Perspectives—Volume" | \'latin-1\' codec can\'t encode character \'\\u2014\' in position 32: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Lecture Notes, http://galton. uchicago. edu/∼ {\\ldots}" | \'latin-1\' codec can\'t encode character \'\\u223c\' in position 44: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Model Regulation Service—October" | \'latin-1\' codec can\'t encode character \'\\u2014\' in position 24: ordinal not in range(256)',
 'tex_to_unicode DECODE ERR | "Sankhy{\\"{A}}\x81: The Indian Journal of Statistics, Series A" | \'ascii\' codec can\'t decode byte 0x81 in position 13: ordinal not in range(128)',
 'tex_to_unicode DECODE ERR | "Sankhyā: The Indian Journal of Statistics" | \'latin-1\' codec can\'t encode character \'\\u0101\' in pos

time: 0 ns (started: 2025-05-19 20:38:29 +01:00)


In [509]:
fGT(jdf.head(100))

index,original,unicoded
0,116 Mich. L. Re.,116 Mich. L. Re.
1,1Mathematical Intelligencer,1Mathematical Intelligencer
2,1Stochastic Processes and their Applications,1Stochastic Processes and their Applications
3,"2003 IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2003. Proceedings.","2003 IEEE Computer Society Conference on Computer Vision and Pattern Recognition, 2003. Proceedings."
4,2012 International Conference on Systems and Informatics (ICSAI2012),2012 International Conference on Systems and Informatics (ICSAI2012)
5,2Variance,2Variance
6,30th Annual Symposium on Foundations of Computer Science,30th Annual Symposium on Foundations of Computer Science
7,31st International Congress of Actuaries,31st International Congress of Actuaries
8,"34th International Conference on Machine Learning, ICML 2017","34th International Conference on Machine Learning, ICML 2017"
9,A Modern Course on Statistical Distributions in Scientific Work,A Modern Course on Statistical Distributions in Scientific Work


time: 203 ms (started: 2025-05-19 20:39:26 +01:00)


In [454]:
fGT(b.df.journal.value_counts().head(100))

journal,count
,"2,643"
Insurance: Mathematics and Economics,145
Journal of Risk and Insurance,136
ASTIN Bulletin,108
SSRN Electronic Journal,60
North American Actuarial Journal,58
Mathematical Finance,50
Risk Management and Insurance Review,46
Journal of Finance,45
The Journal of Finance,44


time: 172 ms (started: 2025-05-19 18:45:14 +01:00)


### Distinct authors

In [844]:
b = sc.Bib2df(p)
adf = b.author_map_df
len(b.df), len(adf), len(adf) / 60, len(b.last_decode)

Decode errors: len(self.last_decode) = 0


(6118, 8160, 136.0, 0)

time: 1.61 s (started: 2025-05-21 07:40:19 +01:00)


In [845]:
b.last_decode

[]

time: 15 ms (started: 2025-05-21 07:40:23 +01:00)


In [852]:
adf.head()

,original,unicoded,initials,proposed,last,rest
0,,,,,,None
1,...,...,...,...,...,None
2,"2017, Undefined","2017, Undefined","2017, Undefined","2017, Undefined",2017,Undefined
3,"A, Hidalgo C{\'{e}}sar","A, Hidalgo César","A, Hidalgo César","A, Hidalgo César",A,Hidalgo César
4,"A, Yildirim Muhammed","A, Yildirim Muhammed","A, Yildirim Muhammed","A, Yildirim Muhammed",A,Yildirim Muhammed


time: 0 ns (started: 2025-05-21 07:43:22 +01:00)


In [851]:
b.author_last_multiple_firsts()

,n,set
last,,
A,2,"[Hidalgo César, Yildirim Muhammed]"
Aas,2,"[K., Kjersti]"
Abraham,5,"[Andreas, Jon, Paul, R., Sarah]"
Acciaio,2,"[B., Beatrice]"
Acemoglu,2,"[By Daron, Daron]"
...,...,...
de Vries,3,"[C., Casper]"
de la Pena,2,"[Victor, Victor H.]"
de la Peña,3,"[Victor, Victor H.]"


time: 2.06 s (started: 2025-05-21 07:42:50 +01:00)


In [899]:
b = sc.Bib2df(p)
print(len(b.df))

uber regex sub found n = 517 replacements


6118

time: 781 ms (started: 2025-05-21 08:51:07 +01:00)


In [901]:
b.port_mendeley_file()

Field: authors
Decode errors: len(self.last_decode) = 0
fixed authors
Field: title
len(self.last_decode) = 37
Fixed title
Field: journal
len(self.last_decode) = 4
Fixed journal
Fixed publisher
Fixed institution
Fixed booktitle
Fixed address
Fixed editor
time: 9.77 s (started: 2025-05-21 08:53:10 +01:00)


In [904]:
b.ported_df.columns

Index(['type', 'tag', 'author', 'doi', 'file', 'journal', 'keywords', 'pages',
       'title', 'volume', 'year', 'publisher', 'url', 'institution', 'number',
       'mendeley-tags', 'booktitle', 'edition', 'month', 'address', 'editor'],
      dtype='object')

time: 0 ns (started: 2025-05-21 08:53:59 +01:00)


In [903]:
b.show_unicode_errors()

{'\x81',
 '\xad',
 '´',
 '¼',
 'ā',
 '‐',
 '†',
 '•',
 '€',
 '™',
 '→',
 '∈',
 '∑',
 '−',
 '∗',
 '∞',
 '∼',
 '≤',
 'ﬁ'}

time: 16 ms (started: 2025-05-21 08:53:45 +01:00)


In [891]:
errs = set([c for l in b.last_decode for c in l if len(c.encode('utf-8')) > 1])
print(errs, len(errs))

{'†', '→', '∑', '™', '´', '−', '∈', '≤', '•', 'ﬁ', '€', '¼', '‐', '∗', '\xad', '∞'} 16
time: 0 ns (started: 2025-05-21 08:34:08 +01:00)


In [865]:
fGT(b.ndf.loc[b.ndf.author.str.contains('Can'), ['author', 'title']])

index,author,title
181,"Marucho, M. and Canal, C. A.Garcia and Fanchiotti, Huner",{The Landau distribution for charged particles traversing thin films}
599,"Canis, Bill",{Motor Vehicle Supply Chain: Effects of the Japanese Earthquake and Tsunami (Google eBook)}
937,"Bellare, M. and Canetti, R. and Krawczyk, H.",{Keying hash functions for message authentication}
944,"Candès, Emmanuel J.",{Mathematics of sparsity (and a few other things)}
1159,"Candès, Emmanuel J.",{Compressive sampling}
1675,"Gangbo, Wilfrid and McCann, Robert J.",{The geometry of optimal transportation}
1727,"Cannas, Giuseppina",{A stochastic model for the sustainable investment policy in a defined benefit pension fund}
1783,Canon,{PowerShot G15 Manual}
1848,"Candès, Emmanuel J. and Wakin, M. B.",{An Introduction To Compressive Sampling}
2241,Canon,{Canon 5DSR Instruction Manual}


time: 46 ms (started: 2025-05-21 07:54:41 +01:00)


In [863]:
fGT(b.df.loc[b.df.author.str.contains('Can'), ['author', 'title']])

index,author,title
181,"Marucho, M. and Canal, C. A.Garcia and Fanchiotti, Huner",{The Landau distribution for charged particles traversing thin films}
599,"Canis, Bill",{Motor Vehicle Supply Chain: Effects of the Japanese Earthquake and Tsunami (Google eBook)}
937,"Bellare, M and Canetti, R and Krawczyk, H",{Keying hash functions for message authentication}
944,"Candes, Emmanuel",{Mathematics of sparsity (and a few other things)}
1159,"Cand{\`{e}}s, Emamnuel J",{Compressive sampling}
1675,"Gangbo, Wilfrid and McCann, Robert J.",{The geometry of optimal transportation}
1727,"Cannas, Giuseppina",{A stochastic model for the sustainable investment policy in a defined benefit pension fund}
1783,Canon,{PowerShot G15 Manual}
1848,"Candes, E.J. and Wakin, M.B.",{An Introduction To Compressive Sampling}
2241,Canon,{Canon 5DSR Instruction Manual}


time: 47 ms (started: 2025-05-21 07:52:26 +01:00)


In [860]:
{k: v for k, v in b.author_mapper().items() if k.startswith('Can')}

{'Canal, C. A.Garcia': 'Canal, C. A.Garcia',
 'Candes, E': 'Candès, Emmanuel J.',
 'Candes, E.': 'Candès, Emmanuel J.',
 'Candes, E.J.': 'Candès, Emmanuel J.',
 'Candes, Emmanuel': 'Candès, Emmanuel J.',
 'Candes, Emmanuel J.': 'Candès, Emmanuel J.',
 'Cand{\\`{e}}s, Emamnuel J': 'Candès, Emmanuel J.',
 'Cand{\\`{e}}s, Emmanuel J': 'Candès, Emmanuel J.',
 'Cand{\\`{e}}s, Emmanuel J.': 'Candès, Emmanuel J.',
 'Canetti, R': 'Canetti, R.',
 'Canis, Bill': 'Canis, Bill',
 'Cann, Matthew': 'Cann, Matthew',
 'Canna, Gabriele': 'Canna, Gabriele',
 'Cannas, Giuseppina': 'Cannas, Giuseppina',
 'Canon': 'Canon',
 'Cantor, Richard': 'Cantor, Richard'}

time: 16 ms (started: 2025-05-21 07:49:05 +01:00)


## Files

In [550]:
b._file_field_df = None
ffdf = b.file_field_df

time: 969 ms (started: 2025-05-19 21:03:13 +01:00)


In [551]:
len(ffdf)

6079

time: 0 ns (started: 2025-05-19 21:03:15 +01:00)


In [553]:
all(ffdf.index == ffdf.idx)

True

time: 0 ns (started: 2025-05-19 21:03:21 +01:00)


In [554]:
ffdf.head()

,idx,drive,file,type
i,,,,
1,1,C\,/S/Library/Schmidt/2003_The true invariant of ...,pdf
2,2,C\,/S/Library/Bahnemann/2015_Distributions for Ac...,pdf
2,2,C\,/S/Library/Bahnemann/2015_Distributions for Ac...,pdf
3,3,C\,/S/Library/Khoshnevisan/2011_Topics in Probabi...,pdf
4,4,C\,/S/Library/Mahul/2003_Efficient Risk Sharing w...,pdf


time: 32 ms (started: 2025-05-19 21:03:23 +01:00)


In [541]:
len(ffdf)

6079

time: 0 ns (started: 2025-05-19 21:00:54 +01:00)


In [560]:
ffdf.groupby(by='idx').file.count().to_frame('count').query('count > 1').sort_values('count')

,count
idx,
2,2
76,2
380,2
420,2
453,2
...,...
2797,3
3077,3
4823,3


time: 31 ms (started: 2025-05-19 21:04:20 +01:00)


In [561]:
_.sum()

count    246
dtype: int64

time: 0 ns (started: 2025-05-19 21:04:33 +01:00)


In [ ]:
missing = []
for i, r in ffdf.iterrows():
    if not Path(r.file).exists():
        print(i, r.file)
        missing.append([i, r.file])

In [526]:
missing[:3]

[[5,
  "/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.'s Non-Adoption of IFRS in 2012.pdf"],
 [37,
  "/S/Library/Filipovi{\\'{c}}, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf"],
 [39,
  "/S/Library/Heras, Balb{\\'{a}}s, Vilar/2012_Conditional tail expectation and premium calculation.pdf"]]

time: 0 ns (started: 2025-05-19 20:44:06 +01:00)


In [525]:
len(missing)

479

time: 0 ns (started: 2025-05-19 20:43:58 +01:00)


In [314]:
ffdf.head().drop(columns='i').file.values

array(['/S/Library/Schmidt/2003_The true invariant of an arbitrage free portfolio.pdf',
       '/S/Library/Bahnemann/2015_Distributions for Actuaries.pdf',
       '/S/Library/Bahnemann/2015_Distributions for Actuaries(2).pdf',
       '/S/Library/Khoshnevisan/2011_Topics in Probability Levy Processes.pdf',
       '/S/Library/Mahul/2003_Efficient Risk Sharing within a Catastrophe Insurance Pool.pdf'],
      dtype=object)

time: 0 ns (started: 2025-05-18 21:50:10 +01:00)


In [533]:
actual_files = [str(i.as_posix()) for i in Path('\\S\\Library').rglob('*.pdf')]
len(actual_files)

6128

time: 2.14 s (started: 2025-05-19 20:54:30 +01:00)


In [534]:
actual_files[:3]

['/S/Library/451 Research/2014_Data Platforms Map.pdf',
 '/S/Library/451 Research/2015_Data Platform Map.pdf',
 "/S/Library/A. M. Best/2004_Best's Insolvency Study, PropertyCasualty U.S. Insurers 1969-2002.pdf"]

time: 16 ms (started: 2025-05-19 20:54:33 +01:00)


In [532]:
missing[:3]

[[5,
  "/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.'s Non-Adoption of IFRS in 2012.pdf"],
 [37,
  "/S/Library/Filipovi{\\'{c}}, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf"],
 [39,
  "/S/Library/Heras, Balb{\\'{a}}s, Vilar/2012_Conditional tail expectation and premium calculation.pdf"]]

time: 0 ns (started: 2025-05-19 20:54:00 +01:00)


In [537]:
ans = []
for i, m in missing:
    best_match = min(actual_files,
        key=lambda alt: Levenshtein.distance(m, alt))
    ans.append([i, m, best_match])

time: 19.7 s (started: 2025-05-19 20:57:50 +01:00)


In [ ]:
ansdf = pd.DataFrame(ans, columns=['idx', 'missing', 'match'])

In [539]:
fGT(ansdf.head(100))

index,idx,missing,match
0,5,"/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.'s Non-Adoption of IFRS in 2012.pdf","/S/Library/Jiang, Koo, Wang/2025_The Road Not Taken Evaluating the Consequences of the U.S.’s Non-Adoption of IFRS in 2012.pdf"
1,37,"/S/Library/Filipovi{\'{c}}, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf","/S/Library/Filipović, Svindland/2008_Optimal capital and risk allocations for law- and cash-invariant convex functions.pdf"
2,39,"/S/Library/Heras, Balb{\'{a}}s, Vilar/2012_Conditional tail expectation and premium calculation.pdf","/S/Library/Heras, Balbás, Vilar/2012_Conditional tail expectation and premium calculation.pdf"
3,43,"/S/Library/M{\""{u}}ller/1998_Comparing risks with unbounded distributions.pdf",/S/Library/Müller/1998_Comparing risks with unbounded distributions.pdf
4,81,"/S/Library/B{\""{u}}hlmann, Jewell/1978_Operational Risk.pdf","/S/Library/Bühlmann, Jewell/1978_Operational Risk.pdf"
5,112,"/S/Library/B{\""{a}}uerle, M{\""{u}}ller/2006_Stochastic orders and risk measures Consistency and bounds.pdf","/S/Library/Bäuerle, Müller/2006_Stochastic orders and risk measures Consistency and bounds.pdf"
6,115,"/S/Library/Garmin - Fenix, Garmin/2013_Owner ' s Manual.pdf","/S/Library/Garmin - Fenix, Garmin/2013_Owner ’ s Manual.pdf"
7,120,"/S/Library/Klartag, Vershynin/2007_Small ball probability and Dvoretzky's Theorem.pdf","/S/Library/Klartag, Vershynin/2007_Small ball probability and Dvoretzky’s Theorem.pdf"
8,125,"/S/Library/K{\""{u}}chler, S{\o}rensen/1994_Exponential families of stochastic processes and L{\'{e}}vy processes.pdf","/S/Library/Küchler, Sørensen/1994_Exponential families of stochastic processes and Lévy processes.pdf"
9,145,/S/Library/Rosi{\'{n}}ski/2011_Infinitely Divisible Processes.pdf,/S/Library/Rosiński/2011_Infinitely Divisible Processes.pdf


time: 234 ms (started: 2025-05-19 20:58:37 +01:00)


In [ ]:
# List of alternatives
alts = ["apple", "banana", "orange", "grape", "pineapple"]

# String to match
s = "applle"

# Find the best match using Levenshtein distance
best_match = min(alts, key=lambda alt: Levenshtein.distance(s, alt))

# Calculate the distance (edit distance)
distance = Levenshtein.distance(s, best_match)

print(f"Best match: {best_match}, Distance: {distance}")

Best match: apple, Distance: 1
time: 125 ms (started: 2025-05-19 20:51:25 +01:00)


# Google Scholar 

In [567]:
import webbrowser
import urllib.parse
import pyperclip

def open_google_scholar(s):
    query = urllib.parse.quote_plus(s)
    url = f'https://scholar.google.com/scholar?q={query}'
    webbrowser.open_new_tab(url)


time: 31 ms (started: 2025-05-20 07:21:54 +01:00)


In [569]:
open_google_scholar('Mildenhall major "pricing insurance risk"')

time: 328 ms (started: 2025-05-20 07:22:20 +01:00)


In [617]:
bibtex = pyperclip.paste()
d = sc.Bib2df.parse_gs(bibtex)
pp.pprint(d)

{'author': 'Mildenhall, Stephen J and Major, John A',
 'publisher': 'John Wiley \\& Sons',
 'tag': 'mildenhall2022pricing',
 'title': 'Pricing insurance risk: Theory and practice',
 'type': 'book',
 'year': '2022'}
time: 0 ns (started: 2025-05-20 07:54:57 +01:00)


In [640]:
def dict_to_bibtex(d):
    """Create Bibtex string from dictionary d."""
    standard_types = ['report', 'article', 'book', 'incollection',
 'inproceedings',
 'manual',
 'misc',
 'phdthesis',
 'techreport',
 'unpublished']
    min_fields = ['type', 'tag', 'title', 'year']
    fails = []
    for f in min_fields:
        if f not in d:
            fails.append(f'missing {f}')
    if fails:
        raise ValueError('Missing required fields\n' + '\n'.join(fails))
    d = d.copy()
    t = d.pop('type')
    tag = d.pop('tag')
    tag = tag[0].upper() + tag[1:]
    title = d.pop('title')
    assert t in standard_types, f'Type must be a standard type not {t}'
    s = f'@{t}{{{tag},\ntitle = {{{{{title}}}}},\n' + '\n'.join(f'{k} = {{{v}}},' for k, v in d.items()) + '\n}'
    return s

time: 15 ms (started: 2025-05-20 07:59:52 +01:00)


In [641]:
print(dict_to_bibtex(d))

@book{Mildenhall2022pricing,
title = {{Pricing insurance risk: Theory and practice}},
author = {Mildenhall, Stephen J and Major, John A},
year = {2022},
publisher = {John Wiley \& Sons},
}
time: 0 ns (started: 2025-05-20 07:59:52 +01:00)


In [633]:
print(b.txt[207:1600])

@article{Schmidt2003,
abstract = {It is shown that the arbitrage free portfolio paradigm being applied to a portfolio with an arbitrary number of shares N allows for the extended solution in which the option price F depends on N. However the resulting stock hedging expense Q = MF (where M is the number of options in the portfolio) does not depend on whether N is treated as an independent variable or as a parameter. Therefore the stock hedging expense is the true invariant of the arbitrage free portfolio paradigm. {\textcopyright} 2002 Elsevier Science B.V. All rights reserved.},
author = {Schmidt, Anatoly B.},
doi = {10.1016/S0378-4371(02)01551-0},
file = {:C\:/S/Library/Schmidt/2003_The true invariant of an arbitrage free portfolio.pdf:pdf},
issn = {03784371},
journal = {Physica A: Statistical Mechanics and its Applications},
keywords = {Arbitrage free portfolio,Option pricing},
pages = {535--538},
title = {{The true invariant of an arbitrage free portfolio}},
volume = {320},
year = {

In [645]:
b = sc.Bib2df(p, True)

time: 703 ms (started: 2025-05-20 08:02:47 +01:00)


In [658]:
fGT(b.df.loc[b.df.annote.str.contains('[Cc]itations', regex=True), ['author', 'title', 'year', 'annote']])

index,author,title,year,annote
3414,"Csoka, Peter and Pinter, Miklos",{On the Impossibility of Fair Risk Allocation},2016,"From Duplicate 2 (On the Impossibility of Fair Risk Allocation - Csoka, Peter; Pinter, Miklos) 27 Google Citations"
3996,"Modigliani, Franco and Miller, Merton H","{The cost of capital, corporate fiance and the theory of investment}",1958,22000 citations
5271,"Camerer, Colin F. and Ho, Teck Hua",{Violations of the betweenness axiom and nonlinearity in probability},1994,755 GSCitations
6028,"Lecun, Yann and Bengio, Yoshua and Hinton, Geoffrey",{Deep learning},2015,"2. ""Deep learning"" (2015) Nature 16,750 citations This paper stands out not just because of its high number of citations, but because there was a difference of more than 10,000 between its citation count and the second most-cited Nature paper in the 2019 Google Scholar Metrics report. Authored by 2018 Turing Award winners, Yann LeCun, Yoshua Bengio and Geoffrey Hinton – known collectively as the 'Godfathers of AI' – the paper is a seminal review of the potential of the AI technologies."


time: 62 ms (started: 2025-05-20 08:10:15 +01:00)


In [ ]:
fGT(b.df.loc[b.df.abstract.str.contains('[Cc]itations?', regex=True), ['author', 'title', 'year', 'abstract']])

In [757]:
# regex to extract group like 1000, 2K, 1,000, 1K-2K etc.
pat = r'([\dKk\- ]+)\+? +(?: Google | GS )?[Cc]itations?'

time: 0 ns (started: 2025-05-20 08:45:38 +01:00)


In [752]:
df = b.df.copy()

time: 0 ns (started: 2025-05-20 08:43:06 +01:00)


In [758]:
v = df.loc[:, 'abstract'].str.extract(pat)[0].str.strip().dropna()
v

124           195
146          6652
228     1000-2500
229          5345
233            66
          ...    
5990     500-1000
5996     500-1000
6035    1000-2500
6036    1000-2500
6108          108
Name: 0, Length: 197, dtype: object

time: 203 ms (started: 2025-05-20 08:45:40 +01:00)


In [759]:
df.loc[2780, 'abstract']

'500+ citations'

time: 0 ns (started: 2025-05-20 08:45:41 +01:00)


In [761]:
re.search(pat, df.loc[2780, 'abstract']).groups(1)

('500',)

time: 0 ns (started: 2025-05-20 08:45:51 +01:00)


In [762]:
def parse_citations(s):
    try:
        return int(s)
    except ValueError:
        pass
        # print('not int')

    try:
        if s.find('-') > 0:
            l, r = s.split('-')        
            l = int(l.strip())
            r = int(r.strip())
            return (l + r) // 2
    except ValueError:
        pass
        # print('not range')

    s = s.lower()
    try:
        if s.find('k') > 0:
            return parse_citations(s.replace('k', '000')) 
    except ValueError:
        # print('not K')
        pass

    return f'Unable to parse {s}'
    # return -99

time: 0 ns (started: 2025-05-20 08:45:53 +01:00)


In [686]:
fGT(df.loc[822, ['title', 'abstract']])

index,822
title,{Identity-based encryption from the Weil pairing}
abstract,7000 GS citations!


time: 47 ms (started: 2025-05-20 08:27:35 +01:00)


In [763]:
w = v.map(parse_citations)
w.head()

124     195
146    6652
228    1750
229    5345
233      66
Name: 0, dtype: object

time: 16 ms (started: 2025-05-20 08:46:01 +01:00)


In [764]:
v[2780]

'500'

time: 16 ms (started: 2025-05-20 08:46:03 +01:00)


In [765]:
df.loc[w[w.str.contains('Unab', regex=True, na=False)].index, ['author', 'year', 'abstract']]

,author,year,abstract
5431,"Bunz, Benedikt and Bootle, Jonathan and Boneh,...",2018,Boneh - 71444 citations...h = 108; Stanford


time: 32 ms (started: 2025-05-20 08:46:05 +01:00)


In [721]:
w[w.str.contains('195', regex=True, na=False)]

Series([], Name: 0, dtype: object)

time: 16 ms (started: 2025-05-20 08:35:23 +01:00)


In [711]:
v[124]

'195 '

time: 15 ms (started: 2025-05-20 08:33:16 +01:00)


In [767]:
b.df.to_csv('dm.csv', encoding='utf-8-sig')

time: 313 ms (started: 2025-05-20 22:42:10 +01:00)


In [768]:
test = pd.DataFrame([(i, chr(i)) for i in range(65, 10000)], columns=['ord', 'char'])

time: 15 ms (started: 2025-05-20 22:44:48 +01:00)


In [770]:
test.to_csv('dm.csv', encoding='utf-8-sig')

time: 16 ms (started: 2025-05-20 22:45:16 +01:00)


In [776]:
test2 = pd.read_csv('dm2.csv', encoding='utf-8-sig', index_col=0)

time: 16 ms (started: 2025-05-20 22:49:19 +01:00)


In [777]:
test2.head()

,ord,char
0,65,A
1,66,B
2,67,C
3,68,D
4,69,E


time: 0 ns (started: 2025-05-20 22:49:20 +01:00)


In [779]:
all(test2==test)

True

time: 0 ns (started: 2025-05-20 22:49:40 +01:00)


In [772]:
fGT(test.head())

index,ord,char
0,65,A
1,66,B
2,67,C
3,68,D
4,69,E


time: 15 ms (started: 2025-05-20 22:48:29 +01:00)


In [689]:
# apply extract and parse
fGT(v.map(parse_citations))

index,0
124,195
146,"6,652"
228,"1,750"
229,"5,345"
233,66
380,196
493,750
501,42
550,114
601,Unable to parse


time: 391 ms (started: 2025-05-20 08:28:21 +01:00)


In [ ]:
fGT(b.df[['tag', 'author', 'year', 'title']].tail(20))

In [612]:
d = {'a': 10, 'b': 20}
d

{'a': 10, 'b': 20}

time: 0 ns (started: 2025-05-20 07:51:22 +01:00)


In [613]:
d.pop('a')

10

time: 0 ns (started: 2025-05-20 07:51:28 +01:00)


In [614]:
d

{'b': 20}

time: 0 ns (started: 2025-05-20 07:51:29 +01:00)


## Bibtex parser read

In [42]:
scbt.LIBRARY_BIB, p, p.exists(), scbt.BOOKS_BIB

('library.bib', WindowsPath('library.bib'), True, 'books.bib')

time: 0 ns (started: 2025-05-17 10:12:17 +01:00)


In [43]:
bdb = scbt.bibtex_read() # too slow... scbt.BOOKS_BIB)

time: 437 ms (started: 2025-05-17 10:12:29 +01:00)


In [44]:
len(bdb)

TypeError: object of type 'BibDatabase' has no len()

time: 47 ms (started: 2025-05-17 10:12:36 +01:00)
